In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yelp-fake-reviews/yelp_fake_reviews.csv


In [2]:
!pip install transformers==4.1.1 sentencepiece==0.1.94
!pip install mosestokenizer==1.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.5/507.5 kB 3.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.2/184.2 kB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 41.6 MB/s eta 0:00:00
  Created wheel for sentencepiece: filename=sentencepiece-0.1.94-cp310-cp310-linux_x86_64.whl size=1151521 sha256=12d2d218a4affc77ea415f8b5a8fe13aa92a0e3d208deec0895c4471ccd7d0f3
  Stored in directory: /root/.cache/pip/wheels/09/a1/54/a3196ca6f241737de6ae3dcdf00a80383f9dd5e5f7dc78a97e
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [47 lines of output]
      running bdist_wheel
      runn

In [3]:
import pandas as pd
import warnings
import numpy as np
from tqdm import tqdm
from transformers import MarianMTModel, MarianTokenizer
from collections import defaultdict

warnings.simplefilter("ignore")

In [4]:
df = pd.read_csv("/kaggle/input/yelp-fake-reviews/yelp_fake_reviews.csv", encoding="latin1")

In [5]:
def map_labels(label):
    if label == 1:
        return 1
    elif label == -1:
        return 0
    else:
        return label
df['LABEL'] = df['LABEL'].apply(map_labels)
#Drop columns which are not necessary
df.drop(["RATING_DEVIATION","RATING_CATEGORY",
         "SINGLE_RATING_CATEGORY","REVIEW_COUNT_DATE",
         "MAX_USER_REVIEWS_DAY",
         "RATIO_POSITIVE_NEGATIVE", "Unnamed: 0", "PERCENTAGE_POSITIVE_REVIEWS"], axis=1, inplace=True)
df.LABEL.apply(map_labels)
df.columns

Index(['USER_ID', 'PRODUCT_ID', 'RATING', 'DATE', 'LABEL', 'REVIEW_TEXT',
       'AVERAGE_RATING', 'TOTAL_PRODUCT_REVIEWS', 'REVIEW_LENGTH',
       'SAME_DATE_MULTIPLE_REVIEWS', 'TIMESTAMP_DIFFERENCE',
       'AVERAGE_USER_REVIEW_LENGTH', 'TOTAL_USER_REVIEWS'],
      dtype='object')

In [6]:
def data_clean(data):
    # total rows in data
    print("Total no. of rows in data:", len(data))

    # checking numbers of NaNs in all columns
    print("Total no. of NaNs in 'RATING' column:", data['RATING'].isnull().sum())
    print("Total no. of NaNs in 'LABEL' column:", data['LABEL'].isnull().sum())
    print("Total no. of NaNs in 'REVIEW_TEXT' column:", data['REVIEW_TEXT'].isnull().sum())

    # removing the rows containing NaNs in 'category' column
    # data = data[data['LABEL'].notna()]
    # print("Total no. of rows in data after removing NaNs:", len(data))

    # count of CG and OR reviews
    print("Count of CG and OR reviews:", data['LABEL'].value_counts())
    print("Ratio of CG and OR REVIEWS:", data['LABEL'].value_counts(normalize=True))

    # avg number of words in review of CG and OR reviews
    data['word_count'] = data['REVIEW_TEXT'].str.split().str.len()
    print("Average count of words in CG and OR reviews:", data.groupby('LABEL')['word_count'].mean())

    return data

In [7]:
data = data_clean(df)

Total no. of rows in data: 20000
Total no. of NaNs in 'RATING' column: 0
Total no. of NaNs in 'LABEL' column: 0
Total no. of NaNs in 'REVIEW_TEXT' column: 0
Count of CG and OR reviews: LABEL
1    17926
0     2074
Name: count, dtype: int64
Ratio of CG and OR REVIEWS: LABEL
1    0.8963
0    0.1037
Name: proportion, dtype: float64
Average count of words in CG and OR reviews: LABEL
0     83.632112
1    121.903269
Name: word_count, dtype: float64


In [8]:
data_OR = data.loc[data['LABEL'] == 1]
data_OR_series = data_OR['REVIEW_TEXT'].to_list()[:6000]

In [9]:
len(data_OR_series)

6000

In [10]:
# functions for back translation (Reference - https://amitness.com/back-translation/)
target_model_name = 'Helsinki-NLP/opus-mt-en-fr'
target_tokenizer = MarianTokenizer.from_pretrained(target_model_name)
target_model = MarianMTModel.from_pretrained(target_model_name).to('cuda:0')

en_model_name = 'Helsinki-NLP/opus-mt-fr-en'
en_tokenizer = MarianTokenizer.from_pretrained(en_model_name)
en_model = MarianMTModel.from_pretrained(en_model_name).to('cuda:0')

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

...

target.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [11]:
def translate(texts, model, tokenizer, language="fr"):
    # Prepare the text data into appropriate format for the model
    template = lambda text: f"{text}" if language == "en" else f">>{language}<< {text}"
    src_texts = [template(text) for text in texts]

    # Tokenize the texts
    encoded = tokenizer.prepare_seq2seq_batch(src_texts,return_tensors="pt").to('cuda:0')

    # Generate translation using model
    translated = model.generate(**encoded)

    # Convert the generated tokens indices back into text
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)

    return translated_texts

def back_translate(texts, source_lang="en", target_lang="fr"):
    # Translate from source to target language
    fr_texts = translate(texts, target_model, target_tokenizer,
                         language=target_lang)

    # Translate from target language back to source language
    back_translated_texts = translate(fr_texts, en_model, en_tokenizer, language=source_lang)

    return back_translated_texts

In [12]:
# breaking augmentation data into batches for easier processing
batch_size = 25 # should be a multiple of length of original data (6000)
num_batches = int(len(data_OR_series) / batch_size)

In [13]:
# function which takes a list as input and returns a dictionary of lists containing the outputs of back translation
def back_translation_output(data):
    i = 0
    d_data = defaultdict(list)
    d_data_out = defaultdict(list)
    for i in tqdm(range(num_batches)):
        d_data[i] = data[i*batch_size:(i+1)*batch_size]
        d_data_out[i] = back_translate(d_data[i])

    return d_data_out

In [14]:
data_CG_bt = back_translation_output(data_OR_series)

100%|██████████| 240/240 [2:06:58<00:00, 31.75s/it]  


In [15]:
data_OR.head()

,USER_ID,PRODUCT_ID,RATING,DATE,LABEL,REVIEW_TEXT,AVERAGE_RATING,TOTAL_PRODUCT_REVIEWS,REVIEW_LENGTH,SAME_DATE_MULTIPLE_REVIEWS,TIMESTAMP_DIFFERENCE,AVERAGE_USER_REVIEW_LENGTH,TOTAL_USER_REVIEWS,word_count
1,74755,449,4,2013-03-26,1,My family and I had Bubby's brunch on a Saturd...,3.396552,812,824,0,1723 days,724.666667,12,156
2,49165,237,3,2011-10-11,1,"I really like this place, but they need to get...",3.799003,602,314,0,0 days,314.000000,1,50
3,75653,363,5,2014-01-14,1,This is one of my favorite places in the US. A...,3.990361,2075,280,0,0 days,280.000000,1,50
4,32402,100,4,2014-12-02,1,Make sure you go with a small group of friends...,3.951812,2677,478,0,398 days,255.666667,3,93
5,14924,50,5,2014-10-12,1,Have been coming here since they opened. The q...,4.351714,671,247,0,0 days,247.000000,1,43


In [16]:
data_CG = data_OR.drop(['LABEL', 'REVIEW_TEXT', 'word_count', 'REVIEW_LENGTH'], axis = 1)

In [17]:
data_CG.head()

,USER_ID,PRODUCT_ID,RATING,DATE,AVERAGE_RATING,TOTAL_PRODUCT_REVIEWS,SAME_DATE_MULTIPLE_REVIEWS,TIMESTAMP_DIFFERENCE,AVERAGE_USER_REVIEW_LENGTH,TOTAL_USER_REVIEWS
1,74755,449,4,2013-03-26,3.396552,812,0,1723 days,724.666667,12
2,49165,237,3,2011-10-11,3.799003,602,0,0 days,314.000000,1
3,75653,363,5,2014-01-14,3.990361,2075,0,0 days,280.000000,1
4,32402,100,4,2014-12-02,3.951812,2677,0,398 days,255.666667,3
5,14924,50,5,2014-10-12,4.351714,671,0,0 days,247.000000,1


In [18]:
data_CG['LABEL'] = 0

In [19]:
data_CG_bt_list = list(data_CG_bt.values())

In [20]:
final_text_CG_list = []
for i in range(len(data_CG_bt_list)):
    for j in range(len(data_CG_bt_list[i])):
        final_text_CG_list.append(data_CG_bt_list[i][j])

In [ ]:
with open('your_file.txt', 'w') as f:
    for line in lines:
        f.write(f"{line}\n")

In [21]:
data_CG['REVIEW_TEXT'] = final_text_CG_list

ValueError: Length of values (6000) does not match length of index (17926)